In [1]:
import sys
import os

# Add the path to `database_scripts` folder
sys.path.append(os.path.abspath("../database_scripts"))
import  get_dataframe as gd
df_merged = gd.merge_dataframe()


Establish SQLite connection.


In [2]:
df_merged.shape

(1413960, 17)

In [3]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC  # For classification; use SVR for regression
from sklearn import  metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import numpy as np

In [4]:
#Normalisierung
scalar = StandardScaler()
X = df_merged.drop(['Wert'], axis = 1)
y = df_merged['Wert']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)
X_test_scaled = scalar.fit_transform(X_test)
X_train_scaled = scalar.fit_transform(X_train)


In [5]:
#püfen RandomForest
#Random Forests can be trained in parallel by setting n_jobs=-1, which utilizes all available CPU cores.
rf_model = RandomForestRegressor(n_estimators=50, random_state= 42,max_depth= 10,n_jobs=-1)
rf_model.fit(X_train_scaled, y_train)
y_pred = rf_model.predict(X_test_scaled)

In [ ]:
#evaluate ranomForest
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
X.columns

In [ ]:

#try to reduce the number of features using PCA
from sklearn.decomposition import PCA
import pandas as pd
X = df_merged.drop(['Wert'], axis = 1)
y = df_merged['Wert']
scalar.fit(X)
scaled_df = scalar.transform(X)
scaled_df
pca = PCA(n_components=2)
pca.fit(scaled_df)
x_pca = pca.transform(scaled_df)

df_pastpca = pd.DataFrame(pca.components_, columns= X.columns)

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
wcss = []
for k in range (1, 11):
    kmeans = KMeans(n_clusters = k)
    kmeans.fit(x_pca)
    wcss.append(kmeans.inertia_)
plt.figure(figsize=(15,10))
plt.grid()
plt.plot(range(1,11), wcss, marker ='x')

In [ ]:
km = KMeans(n_clusters=2)
cluster = km.fit_predict(x_pca)
cluster

In [ ]:
import seaborn as sns
plt.figure(figsize=(15,10))
sns.scatterplot(x = x_pca[ : , 0], y = x_pca[ : , 1], hue= cluster)

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_avg = silhouette_score(x_pca, cluster)
print(f"Silhouette Score: {silhouette_avg:.2f}")

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_pca, y, test_size=0.33, random_state=42)

In [ ]:
#prüfen SupportVector SVC
# Step 2: Set up hyperparameter grid
param_grid = {
    'kernel': ['poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto'],
    'degree': [3, 5],  # For polynomial kernel
}

# Step 3: Perform grid search with cross-validation
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Step 4: Best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Step 5: Evaluate on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate performance

accuracy = metrics.accuracy_score(y_test, y_pred)
print(f"Accuracy on test set: {accuracy}")


In [ ]:
#prüfen KNN
from sklearn.neighbors import KNeighborsRegressor
knn_model = KNeighborsRegressor(n_neighbors=5)  # You can tune k as a hyperparameter
knn_model.fit(X_train, y_train)

# Step 4: Make predictions
y_pred = knn_model.predict(X_test)

# Step 5: Evaluate the model
mae = metrics.mean_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')